In [ ]:
# Google Colab에서 실행할 경우 사용

# drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 파일 경로 지정
import sys
sys.path.append('/content/drive/MyDrive/Workspace/IA_sub_2')
print(sys.path)

import os
os.chdir('/content/drive/MyDrive/Workspace/IA_sub_2')

In [1]:
# 로컬에서 실행할 경우 사용

# 파일 경로 지정
import sys, os
from IPython import get_ipython
notebook_dir = os.path.dirname(os.path.abspath(get_ipython().run_line_magic('pwd', '')))
sys.path.append(notebook_dir)

In [2]:
# 데이터 로드
import pandas as pd
from lib import data_preprocessor

# 데이터 csv를 dataFrame으로 변환
# 데이터 경로는 ../data에 있는 data라는 이름이 들어간 csv 파일로 가정
path = '../data'
dir = os.listdir(path)
for file in dir:
    if 'data' in file:
        data = file
        break
df = data_preprocessor.transform_verilog_results_to_DataFrame(path + '/' + data)
normal_df, data_min, data_max = data_preprocessor.normalize_DataFrame(df)

[] columns are dropped.


In [3]:
# 데이터가 올바르게 로드되었는지 확인
print(normal_df.head(), '\n')
print(data_min, '\n')
print(data_max, '\n')
print(normal_df.shape)

       phig       cit        u0        ua        eu    etamob        up  \
0  0.500000  0.500001  0.500003  0.499997  0.500002  0.500002  0.500001   
1  0.410751  0.511724  0.453817  0.359631  0.202639  0.255747  0.472937   
2  0.745142  0.358529  0.236329  0.538958  0.849749  0.157734  0.099613   
3  0.853537  0.657667  0.574691  0.225739  0.038224  0.912242  0.206496   
4  0.763113  0.319106  0.961664  0.397547  0.107565  0.796154  0.351070   

       rdsw        i0       i62  
0  0.500001  0.222644  0.574051  
1  0.001458  0.263669  0.734298  
2  0.288490  0.079474  0.477440  
3  0.799905  0.115181  0.570204  
4  0.309568  0.176218  0.733957   

phig      4.456108e+00
cit       1.152020e-04
u0        1.434403e-02
ua        1.780053e-01
eu        2.636023e-01
etamob    2.952038e+00
up        4.020056e-10
rdsw      3.850809e+01
i0        8.542756e-13
i62       4.507874e-05
dtype: float64 

phig      4.500892e+00
cit       4.607970e-04
u0        5.737568e-02
ua        7.119981e-01
eu  

In [4]:
# 데이터를 80%의 학습 데이터와 20%의 테스트 데이터로 분할
train_df, test_df = normal_df[:int(len(normal_df)*0.8)], normal_df[int(len(normal_df)*0.8):]

# 데이터를 학습 데이터와 레이블로 분할하여 dataLoader로 변환
input_feature = ['i0', 'i62']
output_feature = ['phig', 'cit', 'u0', 'ua', 'eu', 'etamob', 'up', 'rdsw']
train_dl = data_preprocessor.transform_DataFrame_to_DataLoader(train_df[input_feature], train_df[output_feature], 128, True)

# 테스트 데이터도 동일하게 변환
test_dl = data_preprocessor.transform_DataFrame_to_DataLoader(test_df[input_feature], test_df[output_feature], 1, True)

In [ ]:
# ANN 모델 학습

from lib import model_training
import lib.ANN as ANN
import torch
import torch.nn as nn

ann_model = ANN.ANN(input_feature.__len__(), output_feature.__len__())
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(ann_model.parameters(), lr=1e-5)
ann_model, train_loss, val_loss = model_training.train_model(
    ann_model, 
    train_dl, 
    100,
    criterion,
    optimizer,
    test_dl,
    1,
    True
)

In [16]:
# sklearn을 이용하기 위한 데이터 변환

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터를 학습 데이터와 레이블로 분할
features = normal_df[input_feature]
labels = normal_df[output_feature]

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Random Forest 모델 학습

from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 회귀 모델 생성
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # n_estimators는 트리 개수, 조정 가능

# 모델 학습
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = rf_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# GBM 모델 학습

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

# GBM 모델 생성
gbm_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
multi_output_gbm_model = MultiOutputRegressor(gbm_model)

# 모델 학습
multi_output_gbm_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = multi_output_gbm_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# 1-D CNN 모델 학습

from lib import model_training
import torch
import torch.nn as nn
from lib.CNN import CNN

cnn_model = CNN(input_feature.__len__(), output_feature.__len__())
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-5)
cnn_model, train_loss, val_loss = model_training.train_model(
    cnn_model, 
    train_dl, 
    100,
    criterion,
    optimizer,
    test_dl,
    1,
    True
)

In [17]:
# GPR 모델 학습

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.multioutput import MultiOutputRegressor

# GPR 모델 생성
gpr_model = GaussianProcessRegressor(kernel=RBF() + WhiteKernel(), random_state=42)
multi_output_gpr_model = MultiOutputRegressor(gpr_model)

# 모델 학습
multi_output_gpr_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = multi_output_gpr_model.predict(X_test)

# 모델 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# 모델 저장
import joblib

joblib.dump(rf_model, 'Model/random_forest_model.pkl')

# weight dict 형태로 저장
torch.save(cnn_model.state_dict(), 'Model/cnn_model_weights.pt')
torch.save(ann_model.state_dict(), 'Model/ann_model_weights.pt')
joblib.dump(multi_output_gbm_model, 'Model/gbm_model.pkl')
joblib.dump(multi_output_gpr_model, 'Model/gpr_model.pkl')